[toc]

# XGBoost 调参指南

关于 Python 的 XGBoost 的基本使用可以看：[Python Package Introduction — xgboost 1.2.0-SNAPSHOT documentation](https://xgboost.readthedocs.io/en/latest/python/python_intro.html)

XGBoost 在 Python 中的实现有两种，一种是和 sklearn 结合的实现，感觉这种实现支持的特性会少一些。因此个人不使用 sklearn 的 api。

## 参数

首先，XGBoost 有三类参数可以调节。

Before running XGBoost, we must set three types of parameters: 
- **General parameters** relate to which booster we are using to do boosting, commonly tree or linear model

- **Booster parameters** depend on which booster you have chosen

- **Learning task parameters** decide on the learning scenario. For example, regression tasks may use different parameters with ranking tasks.

### General parameters

1. objective: 
    回归用 reg:squarederror, 分类用 binary:logistic 或 multi:softmax。具体可以看 [XGBoost Parameters — xgboost 1.2.0-SNAPSHOT documentation](https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters)
2. eta: XGBoost的learning rate，$f_t = f_{t-1} + \eta h_t(x)$

### Booster Parameters

Booster Parameters 和具体所选择的 Booster 有关。这里以 gbtree 为例。gbtree 通常为 reglinear 的效果要好。

1. eta [default=0.3]
    - Analogous to learning rate in GBM
    - Makes the model more robust by shrinking the weights on each step
    - Typical final values to be used: 0.01-0.2


3. max_depth [default=6]
    - The maximum depth of a tree, Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample.
    - Should be tuned using CV.
    - Typical values: 3-10

## 调参思路

大的思路是 learning rate 从大到小，然后逐步微调其他参数。

1. 由于树的个数可以用 early-stop 确定，因此不需要调这个参数。因此我们先控制 learning rate 比较大（0.1 左右），此时可以大致确定其他树参数的范围。（为什么一开始控制 learning-rate 比较大？因为大 learning-rate 收敛比较快。我们这个阶段的目的是粗估其他参数的范围，因此不需要很精细）
2. 然后我们在逐步缩小 learning rate, 微调其他的树参数的值。

在调整其他树的参数的值时候，也有一定的顺序。

1. 一般来说，树的深度对模型的影响比较大，因此优先去调。
2. 然后去调树的最大节点数、每个节点的最小样本数等等参数。
3. 正则化参数一般放在后面再调。因为我们是在模型效果比较好，出现过拟合现象时采取使用正则化技术缓解过拟合，来获取在验证集上的提升。因此不需要最开始就调这些参数。

## Python 实例

可以参考 [ 2 ]

### xgb.DMatrix

In [1]:
import xgb
dval = xgb.DMatrix(x_val, y_val)
dtest = xgb.DMatrix(x_test)

ModuleNotFoundError: No module named 'xgb'

### EarlyStopping


```
train(..., evals=evals, early_stopping_rounds=10)
```

# References
1. [XGBoost Parameters — xgboost 1.2.0-SNAPSHOT documentation](https://xgboost.readthedocs.io/en/latest/parameter.html#xgboost-parameters)
2. [Python Package Introduction — xgboost 1.2.0-SNAPSHOT documentation](https://xgboost.readthedocs.io/en/latest/python/python_intro.html#early-stopping)